In [3]:
import pandas as pd
import numpy as np
import datetime
import json
from haversine import haversine
import zipcodes
import os
import paramiko
import logging
import glob
logging.basicConfig(filename='SK_weekly_celery.log', level=logging.INFO)

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

output_path_remote="/home/ruoyi/sk_ta_info/"
output_path_remote_2="/mnt/drv5/sk_internal_data/sk_ta_info/"

my_output_folder_internal="/home/jian/Projects/Smoothie_King/weekly_TA_updates/"



# In[126]:

today_date_Monday=datetime.datetime.now().date()
today_date_Monday=today_date_Monday-datetime.timedelta(days=today_date_Monday.weekday())

str_8_digits=str(today_date_Monday)[:4]+str(today_date_Monday)[5:7]+str(today_date_Monday)[8:10]
# str_8_digits="20190325"


# In[127]:

my_output_folder_internal=my_output_folder_internal+"output_"+str_8_digits+"/"
try:
    os.stat(my_output_folder_internal)
except:
    os.mkdir(my_output_folder_internal)


# In[3]:

# SK Client Access

host = "64.237.51.251" 
port = 22
transport = paramiko.Transport((host, port))

account_SK="SmoothieKing"
pwd_SK="j$uY_e~23#[:9Zz6"

transport.connect(username = account_SK, password = pwd_SK)
sftp = paramiko.SFTPClient.from_transport(transport)


# In[4]:

# email_meassage="Files received: "

new_file_list=sftp.listdir("/mnt/drv5/SmoothieKing")
# new_weekly_store_list=["/home/client/BigLotsData/"+x for x in new_weekly_file_list if Client_Today_NUM_STR in x]
new_weekly_store_list=[x for x in new_file_list if (str_8_digits in x) & ("StoreList" in x)]

if len(new_weekly_store_list)==0:
    print("Store_list_not_received,"+str(datetime.datetime.now()))
elif len(new_weekly_store_list)>1:
    print("Store_list_More_Than_1: "+str(datetime.datetime.now()))
else:
    # email_meassage=email_meassage+str(new_weekly_store_list[0])
    print("Store_list_received")
    store_list_file=new_weekly_store_list[0]
    remote_path="/mnt/drv5/SmoothieKing/"+new_weekly_store_list[0]
    local_path_storelist='/home/jian/Projects/Smoothie_King/store_list_weekly/'+store_list_file
    sftp.get(remote_path,local_path_storelist)


new_weekly_sales=[x for x in new_file_list if (str_8_digits in x) & ("WeeklySalesSummaryData" in x)]

if len(new_weekly_sales)==0:
    print("Store_WeeklySales_not_received,"+str(datetime.datetime.now()))
elif len(new_weekly_sales)>1:
    print("Store_WeeklySales_More_Than_1: "+str(datetime.datetime.now()))
else:
    # email_meassage=email_meassage+" | "+str(new_weekly_store_list[0])
    print("Store_WeeklySales_received")
    store_list_file=new_weekly_sales[0]
    remote_path="/mnt/drv5/SmoothieKing/"+new_weekly_sales[0]
    local_path_sales='/home/jian/Projects/Smoothie_King/Weekly_Sales/'+new_weekly_sales[0]
    sftp.get(remote_path,local_path_sales)

sftp.close()


# In[7]:

import smtplib

from string import Template

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

MY_ADDRESS = 'jubapluscc@gmail.com'
PASSWORD = 'jubaplus2015'

def get_contacts(filename):
    """
    Return two lists names, emails containing names and email addresses
    read from a file specified by filename.
    """

    names = []
    emails = []
    with open(filename, mode='r', encoding='utf-8') as contacts_file:
        for a_contact in contacts_file:
            names.append(a_contact.split()[0])
            emails.append(a_contact.split()[1])
    return names, emails

def read_template(filename):
    """
    Returns a Template object comprising the contents of the 
    file specified by filename.
    """

    with open(filename, 'r', encoding='utf-8') as template_file:
        template_file_content = template_file.read()
    return Template(template_file_content)



# In[9]:

if (len(new_weekly_sales)==1) & (len(new_weekly_sales)==1):

    names, emails = get_contacts('/home/jian/Projects/Smoothie_King/SK_celery/mycontacts_success.txt') # read contacts
    message_template = read_template('/home/jian/Projects/Smoothie_King/SK_celery/message_success.txt')
    print(names, emails, message_template)
    # set up the SMTP server
    s = smtplib.SMTP(host='smtp.gmail.com', port=587)
    s.starttls()
    s.login(MY_ADDRESS, PASSWORD)

    # For each contact, send the email:
    for name, email in zip(names, emails):
        msg = MIMEMultipart()       # create a message

        # add in the actual person name to the message template
        message = message_template.substitute(PERSON_NAME=name.title())

        # Prints out the message body for our sake
        print(message)

        # setup the parameters of the message
        msg['From']=MY_ADDRESS
        msg['To']=email
        msg['Subject']="Smoothie King Data Received"

        # add in the message body
        msg.attach(MIMEText(message, 'plain'))

        # send the message via the server set up earlier.
        # s.send_message(msg)
        del msg

    # Terminate the SMTP session and close the connection
    s.close()

else:
    names, emails = get_contacts('/home/jian/Projects/Smoothie_King/SK_celery/mycontacts_failed.txt') # read contacts
    message_template = read_template('/home/jian/Projects/Smoothie_King/SK_celery/message_failed.txt')
    print(names, emails, message_template)
    # set up the SMTP server
    s = smtplib.SMTP(host='smtp.gmail.com', port=587)
    s.starttls()
    s.login(MY_ADDRESS, PASSWORD)

    # For each contact, send the email:
    for name, email in zip(names, emails):
        msg = MIMEMultipart()       # create a message

        # add in the actual person name to the message template
        message = message_template.substitute(PERSON_NAME=name.title())

        # Prints out the message body for our sake
        print(message)

        # setup the parameters of the message
        msg['From']=MY_ADDRESS
        msg['To']=email
        msg['Subject']="Smoothie King Data Not Received yet"

        # add in the message body
        msg.attach(MIMEText(message, 'plain'))

        # send the message via the server set up earlier.
        s.send_message(msg)
        del msg

    # Terminate the SMTP session and close the connection
    s.close()


Starting_Monday=datetime.date(2019,4,1)
this_monday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday())
diff_days=(this_monday-Starting_Monday).days


if (diff_days/7)%4==0:
    smtpObj = smtplib.SMTP('smtp.gmail.com',587)
    smtpObj.ehlo()
    smtpObj.starttls()
    smtpObj.login('jliangmkt@gmail.com','jlj198256')

    message="""From: Jian Liang <jliangmkt@gmail.com>
To: Jian <jian@jubaplus.com>
MIME-Version: 1.0
Content-type: text
Subject: Bi-weekly Big Lots data transfered

Hi Jian,

"SK running this week, please chech the status later!!!!!!!!!!!!!!"

Thanks,
Jian
        """


    sender="jliangmkt@gmail.com"
    receivers=['jian@jubaplus.com']
    try:
       smtpObj.sendmail(sender, receivers, message)         
       print("Successfully sent email every 4 weeks")
    except:
       print("Error: unable to send email every 4 weeks")




    store_list_df=glob.glob("/home/jian/Projects/Smoothie_King/store_list_weekly/*.xlsx")
    store_list_df=pd.DataFrame({"File_Path":store_list_df})
    store_list_df['Date']=store_list_df['File_Path'].apply(lambda x: os.path.basename(x).split("_")[1].split(".")[0])
    store_list_df['Date']=store_list_df['Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
    store_list_df=store_list_df[store_list_df['Date']>=this_monday-datetime.timedelta(days=21)]
    store_list_df=store_list_df.sort_values("Date",ascending=False)
    if store_list_df.shape[0]!=4:
        logging.info("Error: recent 4 weeks store list count is not 4")
    else:
        store_list_4_weeks=pd.DataFrame()

        for file_path in store_list_df['File_Path'].tolist():
            df=pd.read_excel(file_path,dtype=str)
            df=df[~pd.isnull(df['storenumber'])]
            df=df[df['storenumber']!="nan"]
            df=df[df['status']!="Closed"]
            df=df.drop_duplicates()
            store_list_4_weeks=store_list_4_weeks.append(df)
            store_list_4_weeks=store_list_4_weeks.drop_duplicates("storenumber")

    print("Make Sure 0 as storenumber length>4 (StoreList):",str(len(store_list_4_weeks[store_list_4_weeks['storenumber'].apply(lambda x: len(x))!=4])))

    store_list_4_weeks=store_list_4_weeks.reset_index()
    del store_list_4_weeks['index']

    print(len(store_list_4_weeks['storenumber'].unique().tolist())==len(store_list_4_weeks))
    # Filter later


    # In[7]:


    matched_df_with_SG=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/Celery_SK_TA/QC_Address_With_SG_JL_2019-02-12.xlsx",sheetname="Matched_Stores",dtype=str)
    matched_df_with_SG['SG_zip']=matched_df_with_SG['address_comb_SG'].apply(lambda x: x.split("|")[len(x.split("|"))-1])
    store_zip_dic_updated=matched_df_with_SG.set_index("storenumber").to_dict()['SG_zip']

    store_zip_dic_updated.update({"1152":"70810"})

    store_zip_dic_received=store_list_4_weeks.set_index(['storenumber']).to_dict()['zip']
    def update_zip(x):
        if x in store_zip_dic_updated.keys():
            y=store_zip_dic_updated[x]
        else:
            y=store_zip_dic_received[x]
        return y

    store_list_4_weeks['zip']=store_list_4_weeks['storenumber'].apply(update_zip)
    store_list_4_weeks['zip']=store_list_4_weeks['zip'].apply(lambda x: x.split("-")[0])


    # In[8]:

    sales_data_df=glob.glob("/home/jian/Projects/Smoothie_King/Weekly_Sales/*.xlsx")
    sales_data_df=pd.DataFrame({"File_Path":sales_data_df})
    sales_data_df['Date']=sales_data_df['File_Path'].apply(lambda x: os.path.basename(x).split("_")[1].split(".")[0])
    sales_data_df['Date']=sales_data_df['Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
    sales_data_df=sales_data_df[sales_data_df['Date']>=this_monday-datetime.timedelta(days=21)]
    sales_data_df=sales_data_df.sort_values("Date",ascending=False)
    if sales_data_df.shape[0]!=4:
        logging.info("Error: recent 4 weeks weekly sales count is not 4")
    else:
        sales_data_4_weeks=pd.DataFrame()

        for file_path in sales_data_df['File_Path'].tolist():
            df=pd.read_excel(file_path,dtype=str)
            df['NetSalesSmoothie']=df['NetSalesSmoothie'].astype(float)
            df['NetSalesRetail']=df['NetSalesRetail'].astype(float)
            df=df[(df['NetSalesSmoothie']>0) | (df['NetSalesRetail']>0)]
            df=df.drop_duplicates()
            sales_data_4_weeks=sales_data_4_weeks.append(df)
    sales_data_4_weeks_agg_count=sales_data_4_weeks.groupby(['StoreNumber'])['NetSalesSmoothie'].count().to_frame().reset_index()
    sales_data_4_weeks_agg_count=sales_data_4_weeks_agg_count.rename(columns={"NetSalesSmoothie":"Count_Available"})
    sales_data_4_weeks_agg_count=sales_data_4_weeks_agg_count[sales_data_4_weeks_agg_count['Count_Available']>=3]


    print("Make Sure 0 as storenumber length>4 (Sales):",str(len(sales_data_4_weeks_agg_count[sales_data_4_weeks_agg_count['StoreNumber'].apply(lambda x: len(x))!=4])))
    print(len(sales_data_4_weeks_agg_count['StoreNumber'].unique().tolist())==len(sales_data_4_weeks_agg_count))

    store_list_4_weeks=store_list_4_weeks[store_list_4_weeks['storenumber'].isin(sales_data_4_weeks_agg_count['StoreNumber'].unique().tolist())]
    store_list_4_weeks=store_list_4_weeks.reset_index()
    del store_list_4_weeks['index']
    store_list_4_weeks.shape

    store_GPS_verified=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/QC_address/SK_store_listed_verified_JL_2019-04-29.xlsx",sheetname="assigned_lat_long",dtype=str)
    store_GPS_verified['assigned_lat']=store_GPS_verified['assigned_lat'].astype(float)
    store_GPS_verified['assigned_long']=store_GPS_verified['assigned_long'].astype(float)
    store_GPS_verified['storenumber'].apply(lambda x: len(x)).unique()
    new_store_list=store_list_4_weeks[~store_list_4_weeks['storenumber'].isin(store_GPS_verified['storenumber'].tolist())]
    cols_store_new=new_store_list.columns.tolist()

    force_added_stores=pd.read_excel("/home/jian/Projects/Smoothie_King/SK_celery/SOTF_forced_adding/SOTF_forced_addition_20190429.xlsx",dtype=str)
    new_store_list=new_store_list.append(force_added_stores)
    new_store_list=new_store_list[cols_store_new]

    new_store_list=new_store_list[~new_store_list['storenumber'].isin(store_GPS_verified['storenumber'])]
    if len(new_store_list)>0:
        new_store_list.to_csv(my_output_folder_internal+"new_store_appared.csv",index=False)
    else:
        new_store_list.to_csv(my_output_folder_internal+"no_new_store_detected.csv",index=False)
    print(new_store_list.shape)


    verified_store_lat_dict=store_GPS_verified.set_index("storenumber").to_dict()['assigned_lat']
    verified_store_lng_dict=store_GPS_verified.set_index("storenumber").to_dict()['assigned_long']

    store_list_4_weeks=store_list_4_weeks[~store_list_4_weeks['storenumber'].isin(new_store_list['storenumber'].tolist())]

    store_list_4_weeks['used_lat']=store_list_4_weeks['storenumber'].apply(lambda x: float(verified_store_lat_dict[x]))
    store_list_4_weeks['used_long']=store_list_4_weeks['storenumber'].apply(lambda x: float(verified_store_lng_dict[x]))

Store_list_received
Store_WeeklySales_received
['rgao', 'dhys', 'Jian'] ['rgao@jubaplus.com', 'dhysty90@gmail.com', 'jian@jubaplus.com'] <string.Template object at 0x7fba9736ec18>
Hi,

The 2 weekly files are recieved from SK.

Thanks
Hi,

The 2 weekly files are recieved from SK.

Thanks
Hi,

The 2 weekly files are recieved from SK.

Thanks
Successfully sent email every 4 weeks
Make Sure 0 as storenumber length>4 (StoreList): 0
True
Make Sure 0 as storenumber length>4 (Sales): 0
True
(0, 14)


In [17]:

store_list_4_weeks[pd.isnull(store_list_4_weeks['used_lat'])].to_csv("/home/jian/Projects/Smoothie_King/SK_celery/test/1.csv")

In [14]:
store_list_4_weeks.columns.tolist()

['Address',
 'DriveThru',
 'Franchisee_Name',
 'IPAddress',
 'Latitude',
 'Longitude',
 'Ownership',
 'QC_level',
 'SOTF_Forced',
 'StoreCloseDate',
 'StoreOpenDate',
 'assigned_lat',
 'assigned_long',
 'assigned_source',
 'city',
 'state',
 'status',
 'storenumber',
 'updated_date',
 'used_lat',
 'used_long',
 'zip']

In [15]:
Starting_Monday=datetime.date(2019,4,1)
this_monday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday())
diff_days=(this_monday-Starting_Monday).days

In [16]:
diff_days

28

In [17]:
(diff_days/7)%4

0.0

In [18]:
if (diff_days/7)%4==0:
    smtpObj = smtplib.SMTP('smtp.gmail.com',587)
    smtpObj.ehlo()
    smtpObj.starttls()
    smtpObj.login('jliangmkt@gmail.com','jlj198256')

    message="""From: Jian Liang <jliangmkt@gmail.com>
To: Jian <jian@jubaplus.com>
MIME-Version: 1.0
Content-type: text
Subject: Bi-weekly Big Lots data transfered

Hi Jian,

"SK running this week, please chech the status later!!!!!!!!!!!!!!"

Thanks,
Jian
        """


    sender="jliangmkt@gmail.com"
    receivers=['jian@jubaplus.com']
    try:
       smtpObj.sendmail(sender, receivers, message)         
       print("Successfully sent email every 4 weeks")
    except:
       print("Error: unable to send email every 4 weeks")

Successfully sent email every 4 weeks


In [23]:
store_list_df=glob.glob("/home/jian/Projects/Smoothie_King/store_list_weekly/*.xlsx")
store_list_df=pd.DataFrame({"File_Path":store_list_df})
store_list_df['Date']=store_list_df['File_Path'].apply(lambda x: os.path.basename(x).split("_")[1].split(".")[0])
store_list_df['Date']=store_list_df['Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
store_list_df=store_list_df[store_list_df['Date']>=this_monday-datetime.timedelta(days=21)]
store_list_df=store_list_df.sort_values("Date",ascending=False)

In [24]:
if store_list_df.shape[0]!=4:
    print("Error: recent 4 weeks store list count is not 4")

In [21]:
store_list_df.shape[0]

5

In [22]:
store_list_df

,File_Path,Date
14,/home/jian/Projects/Smoothie_King/store_list_w...,2019-04-29
13,/home/jian/Projects/Smoothie_King/store_list_w...,2019-04-22
12,/home/jian/Projects/Smoothie_King/store_list_w...,2019-04-15
10,/home/jian/Projects/Smoothie_King/store_list_w...,2019-04-08
11,/home/jian/Projects/Smoothie_King/store_list_w...,2019-04-02
